In [1]:
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model, neighbors,datasets
from helpers import *
from sklearn import svm
from cross_validations_lib import *

In [2]:
timestep=0.128040973111396;
fc=1/timestep;

In [3]:
yes_dxy_contents = sio.loadmat('NIRSdxy_yes_signal.mat')
no_dxy_contents = sio.loadmat('NIRSdxy_no_signal.mat')

yes_oxy_contents = sio.loadmat('NIRSoxy_yes_signal.mat')
no_oxy_contents = sio.loadmat('NIRSoxy_no_signal.mat')


In [6]:
def feature_extraction(signal,feature_dictionary):
    feature=[]
    if(feature_dictionary["fft_max_frequencies"]==1):
        f=np.linspace(0, fc, num=signal.size)
        spectrum= np.abs(np.fft.fft(signal));
        fft_max_frequencies=f[np.argsort(spectrum)[-3:]]
        feature=np.concatenate((feature,fft_max_frequencies), axis=0);
    

    if(feature_dictionary["mean"]==1):
        mean=np.mean(signal);
        feature=np.concatenate((feature,[mean]), axis=0)
    
    if(feature_dictionary["variance"]==1):
        variance=np.var(signal)
        feature=np.concatenate((feature,[variance]), axis=0)
    #crest factor 
    if(feature_dictionary["crest_factor"]==1):
        crest_factor=np.sum(np.power(signal,2))/signal.size
        feature=np.concatenate((feature,[crest_factor]), axis=0)
    
    return np.asarray(feature)
#.reshape([feature.size,1])
        
    

In [7]:
def get_train_matrix(channels, feature_dictionary,label):
    list_train=[]
    list_labels=[]
    dim=channels.shape
    for i in range (0,dim[1]):
        single_measurement=channels[0,i]
        dim1=single_measurement.shape
        for j  in range (0,dim1[1]):
            features=feature_extraction(single_measurement[:,j],feature_dictionary)
            list_train.append([features]);
        labels=get_labels(dim1[1],label);
        list_labels.append([labels]);
        
            
            
    train_TX=np.concatenate(list_train)
    labels=np.concatenate(list_labels,axis=1)
    
    return train_TX,labels.T.reshape(labels.size)

def get_labels(number, string):
    if(string=="No"):
        return np.zeros(number)    
    if(string=="Yes"):
        return np.ones(number)
    

# DEOXY SIGNALS

In [8]:
#buildig the train matrix and labels

channels_no=no_dxy_contents["no_signal"]
channels_yes=yes_dxy_contents["yes_signal"]

# select which feature select
feature_dictionary = {
        "fft_max_frequencies" : 1, 
         "mean" : 1, 
         "variance" : 1,
         "crest_factor" : 1
         }



train_TX_yes_deoxy,labels_yes=get_train_matrix(channels_yes, feature_dictionary,"Yes");
train_TX_no_deoxy,labels_no=get_train_matrix(channels_no, feature_dictionary,"No");


train_TX=np.concatenate((train_TX_yes_deoxy,train_TX_no_deoxy,),axis=0)

print(train_TX.shape)
labels=np.concatenate((labels_yes,labels_no),axis=0)

(900, 6)


# Logistic Regression

In [7]:
# cross validation

degrees = range(2,4)
lambdas = np.logspace(-8,-2,6)
k_fold=2
seed = 2
max_iters = 5000
print(train_TX.shape)

best_deg,best_lambda, best_accuracy_test, corresponding_accuracy_train = \
        cross_validation_logistic_regularized(labels,train_TX, degrees, lambdas, k_fold, seed, max_iters)


(900, 6)
--- Fold 0 ---
++ Degree 2 ++
>> Lambda 1e-08 <<
0.464444444444
>> Lambda 1.58489319246e-07 <<
0.464444444444
>> Lambda 2.51188643151e-06 <<
0.464444444444
>> Lambda 3.98107170553e-05 <<
0.464444444444
>> Lambda 0.00063095734448 <<
0.464444444444
>> Lambda 0.01 <<
0.471111111111
++ Degree 3 ++
>> Lambda 1e-08 <<
0.471111111111
>> Lambda 1.58489319246e-07 <<
0.471111111111
>> Lambda 2.51188643151e-06 <<
0.471111111111
>> Lambda 3.98107170553e-05 <<
0.471111111111
>> Lambda 0.00063095734448 <<
0.471111111111
>> Lambda 0.01 <<
0.471111111111
--- Fold 1 ---
++ Degree 2 ++
>> Lambda 1e-08 <<
0.466666666667
>> Lambda 1.58489319246e-07 <<
0.466666666667
>> Lambda 2.51188643151e-06 <<
0.466666666667
>> Lambda 3.98107170553e-05 <<
0.466666666667
>> Lambda 0.00063095734448 <<
0.466666666667
>> Lambda 0.01 <<
0.466666666667
++ Degree 3 ++
>> Lambda 1e-08 <<
0.466666666667
>> Lambda 1.58489319246e-07 <<
0.466666666667
>> Lambda 2.51188643151e-06 <<
0.466666666667
>> Lambda 3.98107170553e-

In [8]:
#logistic regression
train_TX = add_powers(train_TX,best_deg )
train_TX= add_ones(train_TX)
logreg=linear_model.LogisticRegression(C=1/best_lambda, class_weight="balanced",max_iter=max_iters)
logreg.fit(train_TX,labels)
predicted_labels= logreg.predict(train_TX)
logreg_accuracy=get_accuracy(predicted_labels, labels)
print(logreg_accuracy)





0.4811111111111111


# KNN

In [ ]:
#KNN 
clf = neighbors.KNeighborsClassifier(200)
clf.fit(train_TX, labels)
predicted_labels= clf.predict(train_TX)
KNN_accuracy=get_accuracy(predicted_labels, labels)
print(KNN_accuracy)



# SVM

In [ ]:
degrees = range(2,5)
C_parameters= np.linspace(0.1,4,2)
k_fold=3 # number of k sub-folders to divide the set
seed = 2
max_iters = 10000
kernel_types=['linear', 'rbf', 'sigmoid']
print(train_TX.shape)

best_deg,best_C, best_kernel_type ,best_accuracy_test, corresponding_accuracy_train = \
        cross_validation_SVM(labels,train_TX, degrees, C_parameters, kernel_types, k_fold, seed, max_iters)



(900, 12)
--- Fold 0 ---
++ Degree 2 ++
>> Penalty parameter C 0.1 <<
>> Type of Kernel  linear <<
0.466666666667 0.523333333333
>> Type of Kernel  rbf <<
0.45 0.53
>> Type of Kernel  sigmoid <<
0.526666666667 0.475
>> Penalty parameter C 4.0 <<
>> Type of Kernel  linear <<
0.46 0.535
>> Type of Kernel  rbf <<
0.49 0.546666666667
>> Type of Kernel  sigmoid <<
0.526666666667 0.461666666667
++ Degree 3 ++
>> Penalty parameter C 0.1 <<
>> Type of Kernel  linear <<


In [12]:
from sklearn import svm

train_TX = add_powers(train_TX,best_deg )


clf = svm.SVC(C=best_C, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel=best_kernel_type,
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
clf.fit(train_TX, labels)  
predicted_labels= clf.predict(train_TX)
SVM_accuracy=get_accuracy(predicted_labels, labels)
print(SVM_accuracy)

0.48333333333333334
